(practical-application)=
# 3.3 Practical Application of Numerical Derivatives - Calculation the end effector velocity based on the joint velocities

We can measure the joint velocities of our Openmanipulator. However, we are interested in the speed of the end effector. We have seen in the lecture that we can calculate the end effector velocity based on the joint velocities using the Jacobian matrix. 

## Estimating the Jacobian matrix of the Openmanipulator-X kinematics
To estimate the velocity of the end effector we need to know the forward kinematics of the Openmanipulator-X and estimate the derivative of the forward kinematics. 



In [ ]:
import Pkg
Pkg.activate("../Exercise 3/OpenMEnv") 
using Revise
include("../OpenManipulatorLib/OpenManipulatorKinematics.jl")
using .OpenManipulatorKineamtics

Estimate the jacobian matrix of the forward kinematics for our robot arm using automatic differentiation. We can use the function `ForwardDiff.jacobian` to calculate the Jacobian matrix of the forward kinematics. 

In [ ]:
using ForwardDiff

function openmanipulator_jacobian(q)
    # Define a function that returns the position of the end effector
    f(q) = OpenManipulatorKineamtics.complete_forward_kinematics(q)[1]

    # Calculate the Jacobian matrix using ForwardDiff.jacobian
    J = ForwardDiff.jacobian(f, q)

    return J
end

Now we can use the function `ReverseDiff.jacobian` to calculate the Jacobian matrix of the forward kinematics.

In [ ]:
using ReverseDiff

function openmanipulator_jacobian_rev_AD(q)
    # Define a function that returns the position of the end effector
    f(q) = OpenManipulatorKineamtics.complete_forward_kinematics(q)[1]

    # Calculate the Jacobian matrix using ForwardDiff.jacobian
    J = ReverseDiff.jacobian(f, q)

    return J
end

Now estimate the jacobian matrix using finite differences:

In [ ]:
import FiniteDifferences

function openmanipulator_jacobian_finiteDiff(q)
    # Define a function that returns the position of the end effector
    f(q) = OpenManipulatorKineamtics.complete_forward_kinematics(q)[1]
    
    J = FiniteDifferences.jacobian(FiniteDifferences.central_fdm(2, 1), f, q)[1]

    return J
end

Now, lets compare the time and results of the methods. What do you observe?

In [ ]:
q = [0.1, 0.0, 0.0, 0.1]

In [ ]:
@time openmanipulator_jacobian(q)

In [ ]:
@time openmanipulator_jacobian_rev_AD(q)

In [ ]:
@time openmanipulator_jacobian_finiteDiff(q)

## Calculating the end effector velocity using the Jacobian matrix

Erlier we said that based on the total derivative we can derive the rule for differentiating our system of equations $f_1(x,y)$ and $f_2(x,y)$ with respect to the variable $t$ when $x$ and $y$ are functions of $t$:

$$
\frac{d}{dt} \begin{bmatrix}
f_1(x,y) \\
f_2(x,y)
\end{bmatrix} = \begin{bmatrix}
\frac{\partial f_1}{\partial x} & \frac{\partial f_1}{\partial y} \\
\frac{\partial f_2}{\partial x} & \frac{\partial f_2}{\partial y}
\end{bmatrix} \begin{bmatrix}
\frac{\partial x}{\partial t} \\
\frac{\partial y}{\partial t}
\end{bmatrix}
= \mathbf{J} \begin{bmatrix}
\frac{\partial x}{\partial t} \\
\frac{\partial y}{\partial t}
\end{bmatrix}
$$

Based on this rule we can estimate the end effector velocity using the Jacobian matrix. We can use the following estimation:

$$
v = \mathbf{J}(q) * \mathbf{q}_v = \mathbf{J}(q) * \begin{bmatrix}
\frac{\partial q_1}{\partial t} \\
\frac{\partial  q_2}{\partial t} \\
\frac{\partial  q_3}{\partial t} \\
\frac{\partial  q_4}{\partial t} \\
\end{bmatrix} 
\approx J(q) * \begin{bmatrix}
\Delta q_1 \\
\Delta q_2 \\
\Delta q_3 \\
\Delta q_4 \\
\end{bmatrix} 
$$

where $v$ is the end effector velocity, $\mathbf{J}(q)$ is the Jacobian matrix evaluated at the current joint angles $q$, and $\mathbf{q}_v$ is the joint velocity vector, which we can estimate with the joint approximate velocities $\Delta q_1$, $\Delta q_2$, $\Delta q_3$, and $\Delta q_4$.

Implement the `endeffector_velocity` function that uses the Jacobian matrix to calculate the end effector velocity. The function should takes in the joint angles `q` and joint velocity vector `q_v`, and returns the end effector velocity `v` calculated using the Jacobian matrix. Use the `openmanipulator_jacobian` function you implemented to calculate the Jacobian matrix.

In [ ]:
function endeffector_velocity(q, q_v)
    # Calculate the Jacobian matrix at the current joint angles
    J = openmanipulator_jacobian(q)

    # Calculate the end effector velocity using the Jacobian matrix
    v = J * q_v

    return v
end

Let's test the function with the following joint angles and joint velocities:

In [ ]:
q_v = [0.0, 0.0, 0.0, 0.0]
v = endeffector_velocity(q, q_v)

## Applying the end effector velocity calculation to the Openmanipulator-X

- let's start the Openmanipulator and subscrib to the joint states
- Calculate the end effector velocity using the joint velocities and the Jacobian matrix
- Do this by adding a new function which is the new callback function for the joint states subscriber and which calls the `endeffector_velocity` function
- print the end effector velocity to the console while you move the Openmanipulator

Since we are working with subscriptions it might be handy to be ablte to restart the notebook withour restarting Gazebo. Go to your Terminal and start the following two programs in seperate cells:

```bash
roslaunch open_manipulator_gazebo open_manipulator_gazebo.launch
```

```bash
roslaunch open_manipulator_controller open_manipulator_controller.launch use_platform:=false
```



In [ ]:
include("../OpenManipulatorLib/OpenManipulatorBase.jl")
using .OpenManipulatorBase

In [ ]:
OpenManipulatorBase.init()

In [ ]:
function sub_joint_to_endeffector(list)
    OpenManipulatorBase.reset_robot()
    OpenManipulatorBase.rossleep(0.5)

    function joint_state_sub_callback(sensor_msg::JointState, a::Array{Any,1})
        header_time = to_sec(sensor_msg.header.stamp)/60.0 # header time in minutes
        names = sensor_msg.name
        positions = sensor_msg.position
        velocities = sensor_msg.velocity
        # v = J * q_v
        joint_names = names[3:6]
        q_v = velocities[3:6]
        q = positions[3:6]
        v = endeffector_velocity(q, q_v)
        println("Header Time: $header_time minutes")
        println("Name: $joint_names")
        println("Joint Velocities: $q_v")
        println("Joint Angles: $q")
        println("End Effector Velocity: $v")
        push!(a, v)
        if length(a) >= 2
            unsubscribe(sub)
        end
    end

    sub = Subscriber{JointState}("/joint_states", joint_state_sub_callback, (list,), queue_size=1)

    sending_time = to_sec(RobotOS.now())/60.0
    println("Sending Time: $sending_time minutes")
    OpenManipulatorBase.set_joint_angles([0.6, 0.6, -0.5, -0.1])
    after_sending_time = to_sec(RobotOS.now())/60.0
    println("After sending Time: $after_sending_time minutes")
    OpenManipulatorBase.rossleep(1.0)
    return list
end

In [ ]:
list = []
list = sub_joint_to_endeffector(list)
sleep(3.0)
println("End Effector Velocities: $list")